# Data loading and exploratory analysis

## Data loading and import modules

In [1]:
import pandas as pd
import numpy as np
import pandas_profiling

from data_utils.load import beautiful_head, get_data, get_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import KFold

/Users/kp/anaconda/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('../data/train.csv', sep=';')
test = pd.read_csv('../data/test.csv', sep=';')

/Users/kp/anaconda/envs/python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# check null value existance
print train.isnull().sum().any()
print test.isnull().sum().any()

True
True


In [4]:
# replace NaN values with zero
train.fillna(value=0, inplace=True)
test.fillna(value=0, inplace=True)

In [5]:
# check one more time null value existance
print train.isnull().sum().any()
print test.isnull().sum().any()

False
False


## Labels encoding

In [6]:
def onehotencoding(data, features='all'):
    #функция для перевода всех категориальных данных в числа с помощью кодировки
    #import necessary modules
    from sklearn.preprocessing import LabelEncoder, OneHotEncoder
    import numpy as np
    import pandas as pd
    
    #write only categorical names to variable cat_columns
    #записать названия столбцов, в которых хранятся категориальные переменные
    #из всех фич выбрать только категориальные
    # <YOUR CODE>
   
    # create new dataframe
    # <YOUR CODE>
    new_data = pd.DataFrame(pd.get_dummies(data[cat_columns],columns=data[cat_columns].columns,sparse=False))

    
    
    return new_data

In [7]:
# prepare data for label encoding

# first save target in a separate variable and drop index, id, codepostal
# <YOUR CODE>
target_train = pd.DataFrame(train['prime_tot_ttc'])
train.drop(['id','codepostal','prime_tot_ttc'], axis = 1, inplace=True)
test.drop(['id','codepostal'], axis = 1, inplace=True)
train.reset_index(drop=True)
test.reset_index(drop=True)
# stack train and test sets for label encoding
# <YOUR CODE>

stack = pd.concat([train,test])
#find categorical variables for encoding
# <YOUR CODE>
cat_columns = list(stack.select_dtypes(include=['category','object']))

RangeIndex(start=0, stop=300000, step=1)
RangeIndex(start=0, stop=30000, step=1)


In [8]:
# apply ohe function
stack = pd.concat([stack, onehotencoding(stack, cat_columns)], axis=1)
stack.drop(cat_columns, axis = 1, inplace=True)




In [9]:
# devide back new data frame on train and test sets
train = stack.head(300000)
test = stack.tail(3000)

In [10]:
# you should get train set shape is (300000, 266) and test set shape is (30000, 266)
print len(train.index)
print len(train.columns.values)
print len(test.index)
print len(test.columns.values)
print len(target_train.index)
print len(target_train.columns.values)

300000
266
3000
266
300000
1


## Modeling and CV

In [11]:
#initializing kfold for our cross validation
from sklearn.cross_validation import KFold

kf = KFold(len(train.index),n_folds=3)

In [12]:
# metric mape
def mape(y_true,y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.mean([np.abs(i - j)/j for i,j in zip(y_true,y_pred)]) *100

In [13]:
# apply cv and train RandomForestRegressor model
"""for train_index, test_index in kf:
    print("TRAIN:", train_index, "TEST:", test_index)
    regressor = RandomForestRegressor()
    regressor.fit(train.ix[train_index],target_train.ix[train_index].values.ravel())
    predicted_train = regressor.predict(train.ix[test_index])
    print mape(target_train.ix[test_index], predicted_train)"""


'for train_index, test_index in kf:\n    print("TRAIN:", train_index, "TEST:", test_index)\n    regressor = RandomForestRegressor()\n    regressor.fit(train.ix[train_index],target_train.ix[train_index].values.ravel())\n    predicted_train = regressor.predict(train.ix[test_index])\n    print mape(target_train.ix[test_index], predicted_train)'

## Make predictions

In [14]:
#run randomforest on the whole train set
# <YOUR CODE>
regressor = RandomForestRegressor()
regressor.fit(train,target_train.values.ravel())



/Users/kp/anaconda/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [15]:
#make predictions
# <YOUR CODE>
predicted_test = regressor.predict(test)

# save predictions

In [19]:
pd.DataFrame(predicted_test).to_csv('../data/result_test.csv', sep = ';')